In [1]:
# Import Required Packages
%matplotlib inline
import numpy as np
#import networkx as nx
import time
import random
import json
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from sympy import *
from tqdm.notebook import tqdm

#From Qiskit library
from qiskit import *
from qiskit.visualization import *
from qiskit import IBMQ
from qiskit import BasicAer
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import Statevector
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.circuit import Parameter
from qiskit.optimization.converters import IsingToQuadraticProgram
from qiskit.aqua.components.optimizers import NELDER_MEAD, CRS, ADAM

In [2]:
qiskit.qiskit.__qiskit_version__

{'qiskit-terra': '0.16.1',
 'qiskit-aer': '0.7.1',
 'qiskit-ignis': '0.5.1',
 'qiskit-ibmq-provider': '0.11.1',
 'qiskit-aqua': '0.8.1',
 'qiskit': '0.23.1'}

In [3]:
#Function: Load JSON file
def loadjson( filename ):
    loaded_file = open(filename, "r")
    json_data = json.load(loaded_file)
    loaded_file.close()
    return json_data

#Function: Load Pickle file
def loadpkl( filename ):
    loaded_file = open(filename, "rb")
    pkl_data = pickle.load( loaded_file )
    loaded_file.close()
    return pkl_data

#Function: Get single term coefficients (to 4 decimal places) from Sympy Expression 
def get_coeff( variable, expression ):
    poly = Poly(expression.coeff(variable), domain = 'RR')
    return round(float(poly.coeffs()[len(poly.coeffs())-1]), 4)

#Function: Get coefficient of quadratic term (to 4 decimal places) from Sympy Expression
def quad_coeff( var_1, var_2, expression ):
    return round(float(expression.coeff(var_1).coeff(var_2)), 4)

In [4]:
pwd

'/home/jedwin/trafficQAOA'

In [5]:
import sys
sys.executable

'/home/jedwin/anaconda3/envs/QC/bin/python'

In [6]:
#Read and store files into variables
car_routes = loadjson( "3cars3routes.json" )
# H = loadjson( "Hamiltonian9qbits.json")
# [objective, Ham, objective_normed, Ham_normed] = loadpkl("expressions.pkl")
[total_cost, constraints] = loadpkl("cost_constraints.pkl")

In [7]:
Q = IndexedBase('Q')
car_routes_variables = {}
for route in car_routes:
    car_routes_variables[route] = Q[route[3],route[10]]
car_routes_variables

{'car1_route1': Q[1, 1],
 'car1_route2': Q[1, 2],
 'car1_route3': Q[1, 3],
 'car2_route1': Q[2, 1],
 'car2_route2': Q[2, 2],
 'car2_route3': Q[2, 3],
 'car3_route1': Q[3, 1],
 'car3_route2': Q[3, 2],
 'car3_route3': Q[3, 3]}

In [8]:
no_routes = 3
no_cars = 3
no_qubits = no_routes * no_cars

In [9]:
#Associate binary variable to each Q11, Q12, etc with one index for QuadraticProgram class
binary_vars = {}
for variable in car_routes_variables:
    binary_vars[car_routes_variables[variable]] = 'X' + str(no_routes*(int(variable[3])-1)+int(variable[10]))
Qvariables = tuple(binary_vars.keys())
Xvariables = tuple(binary_vars.values())
print(binary_vars)

{Q[1, 1]: 'X1', Q[1, 2]: 'X2', Q[1, 3]: 'X3', Q[2, 1]: 'X4', Q[2, 2]: 'X5', Q[2, 3]: 'X6', Q[3, 1]: 'X7', Q[3, 2]: 'X8', Q[3, 3]: 'X9'}


In [10]:
#Construct Quadratic Problem using known (quadratic) objective function
penalty = 132*1.2
qubo1 = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo1.binary_var( binary_vars[variable] )
    linear.append( get_coeff(variable, total_cost) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        quadratic[pairing] = coeff
qubo1.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo1.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty)
qubo1 = converter.convert(qubo1)

In [11]:
op, offset = qubo1.to_ising()

In [12]:
coeffs = [abs(op.oplist[i].coeff) for i in range(len(op.oplist))]
norm_factor = np.amax(coeffs)

In [13]:
#Normalised objective function
qubo = QuadraticProgram()
linear = []
for variable in car_routes_variables.values():
    qubo.binary_var( binary_vars[variable] )
    linear.append( (get_coeff(variable, total_cost)/norm_factor) )
quadratic = {}
for i in range(no_qubits):
    for j in range(i+1, no_qubits):
        pairing = ( Xvariables[i], Xvariables[j] )
        coeff = quad_coeff( Qvariables[i], Qvariables[j], total_cost )
        quadratic[pairing] = (coeff/norm_factor)
qubo.minimize(linear = linear, quadratic = quadratic)

#Add constraints
for car in constraints:
    i = 3*(int(car[3])-1) #to specify index of variable on next line
    qubo.linear_constraint(linear={'X'+str(i+1): 1, 'X'+str(i+2): 1, 'X'+str(i+3): 1}, sense='==', rhs=1, name=str(car))

#Convert constrained Quadratic Problem into QUBO
converter = QuadraticProgramToQubo(penalty = penalty/norm_factor)
qubo = converter.convert(qubo)

In [14]:
op, offset = qubo.to_ising()

In [15]:
qaoa_mes = QAOA(quantum_instance=Aer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [16]:
%%time
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: 1.2999467234949391
optimal value: [0. 1. 0. 0. 1. 0. 1. 0. 0.]
status: SUCCESS
CPU times: user 58.2 ms, sys: 0 ns, total: 58.2 ms
Wall time: 57 ms


In [17]:
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    return qubo_problem.objective.evaluate([int(x[no_qubits-1-i]) for i in range(no_qubits)])

In [18]:
print(1e3)

1000.0


In [79]:
simulator = StatevectorSimulator(method = "statevector_gpu")
def find_optimal_params( no_layers, qubo_problem ):
    optimizer = NELDER_MEAD(maxiter = 1)
    initial_freq_params = [ 0.64438965, -0.65847032]
    op, offset = qubo_problem.to_ising()
    #perturb optimal solution n times:
    no_perturb=5
    perturb_scale_factor = 0.6
    generator = np.random.default_rng()
    for k in range(no_layers):
        p=k+1 #Start indexing from p=1.
        variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
        print(f'Step {p}: Building circuit...')
        circuit = construct_QAOAcirc(p, op)
        print(f'Step {p}: Now optimising parameters (Nelder-Mead)...')
        obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) #Define obj_function that inputs only parameters
        #Step 1
        if p == 1:
            print('initial freq point: '+str(initial_freq_params))
            optimal_result = optimizer.optimize(num_vars = 2*p,
                                                objective_function = obj_expectation,
                                                variable_bounds= variable_bounds, 
                                                initial_point=initial_freq_params)
            optimal_params_B, exp_val_B, no_evals = optimal_result
            optimal_params_L, exp_val_L = optimal_params_B, exp_val_B
            
        #All other steps with random perturbations 
        else:
            optimal_params_L, optimal_params_B = find_perturbed(pp_optimal_params_L, optimizer, p, obj_expectation, variable_bounds, pp_optimal_params_B)
            
        #If not yet on the last layer, add another layer of parameters initialised to 0.05, and same for randomly perturbed params 
        if p != no_layers:
            pp_optimal_params_L = np.zeros(shape = (len(optimal_params_L)+2,), dtype = float)+0.05
            pp_optimal_params_B_0 = pp_optimal_params_L
            
            pp_optimal_params_L[0:p] = optimal_params_L[0:p]
            pp_optimal_params_L[p+1:2*p+1] = optimal_params_L[p:2*p]
            
            pp_optimal_params_B_0[0:p] = optimal_params_B[0:p]
            pp_optimal_params_B_0[p+1:2*p+1] = optimal_params_B[p:2*p]
            
            pp_optimal_params_B = np.empty(shape = (no_perturb+1, ), dtype = object)
            pp_optimal_params_B[0] = pp_optimal_params_B_0
            
            u_random = np.array([np.array(generator.normal(0, optimal_params_B[i]**2, no_perturb+1)) for i in range(0,p)])
            v_random = np.array([np.array(generator.normal(0, optimal_params_B[i]**2, no_perturb+1)) for i in range(p,2*p)])
            
            for j in range(1,len(pp_optimal_params_B)):
                params=np.zeros(shape = (len(optimal_params_B)+2,), dtype=float)+0.05
                params[0:p] = pp_optimal_params_B_0[0:p] + perturb_scale_factor*u_random[:,j]
                params[p+1:2*p+1] = pp_optimal_params_B_0[p+1:2*p+1] + perturb_scale_factor*v_random[:,j]
                pp_optimal_params_B[j] = params
            print("\nlocal params: ", pp_optimal_params_L)   
            print("perturbed params: ", pp_optimal_params_B)
                
        #If on the last layer, choose best parameters 
        else:
            optimal_params = optimal_params_B
        print(f'Step {p}: Done.')
    return optimal_params

#Construct QAOA circuit of size p given operator from qubo problem ( operator is obtained from QuadraticProblem.to_ising() )
def construct_QAOAcirc( p, operator ):
    #Initialise parameter list
    params_expr = []
    
    #Create arbitrary "gamma_n" parameters
    for a in range(p):
        params_expr.append( Parameter('a'+str(a+1)) )
    #Likewise for "beta_n" parameters
    for b in range(p):
        params_expr.append( Parameter('b'+str(b+1)) )
        
    #Create circuit given operator, gamma, and beta parameters
    qaoa = QAOA(operator = operator, p=p, quantum_instance=Aer.get_backend('statevector_simulator'))
    circuit = qaoa.construct_circuit( params_expr[0:2*p] )[0]
    return p, params_expr, circuit

def expectation( qaoa_circuit, qubo_problem, freq_params ):
    #unload p, params_expr, circuit from the return value of contruct_QAOAcirc
    p = qaoa_circuit[0]
    params_expr = qaoa_circuit[1]
    circuit = qaoa_circuit[2]
    params = convert_fourier_parameters(p,p,freq_params)
    if len(params) != 2*p:
        print("Must have len(params) = 2*no_layers")
    else:    
        exp_val = 0
        circuit2 = circuit.assign_parameters({params_expr[i]: params[i] for i in range(len(params))}, inplace = False)
        result = execute(circuit2, simulator).result()
        statevector = Statevector(result.get_statevector())
        prob_dict = statevector.probabilities_dict(decimals = 8)
        for state in prob_dict:
            exp_val += np.round(prob_dict[state], 5) * np.round(eval_cost(state, qubo_problem), 5)
        print(".", end = "")
        return np.round(exp_val, 5)
    
def convert_fourier_parameters(p , q, frequency_parameters):
    u_params = np.array(frequency_parameters[0:q], dtype = float)
    v_params = np.array(frequency_parameters[q:2*q], dtype = float)
    gamma_params = np.empty((p,), dtype = np.float32)
    beta_params = np.empty((p,), dtype = np.float32)
    for i in range(p):
        gamma_inter, beta_inter = 0,0
        a = (i+0.5)*np.pi/p
        for k in range(q):
            gamma_inter += u_params[k]*np.sin( (k+0.5)*a )
            beta_inter += v_params[k]*np.cos( (k+0.5)*a )
        gamma_params[i] = gamma_inter
        beta_params[i] = beta_inter
    return np.append(gamma_params, beta_params)
    
#Binary string x of length 9, using cost/obj function from QUBO on QuadraticProgram qp.
def eval_cost(x, qubo_problem):
    return qubo_problem.objective.evaluate([int(x[no_qubits-1-i]) for i in range(no_qubits)])

def find_perturbed(pp_optimal_params_L, optimizer, p, obj_expectation, variable_bounds, pp_optimal_params_B):
    print('initial freq point L: ', pp_optimal_params_L)
    optimal_result = optimizer.optimize(num_vars = 2*p,
                            objective_function = obj_expectation,
                            variable_bounds= variable_bounds, 
                            initial_point=pp_optimal_params_L)
    optimal_params_L, exp_val_L = optimal_result[0:2]

    perturbed_params = [optimal_params_L]
    perturbed_exp_vals = [exp_val_L]
    for l in range(len(pp_optimal_params_B)):
        print('\n'+f'{l}, initial freq point B: ', pp_optimal_params_B[l])

        perturbed_optimal_result = optimizer.optimize(num_vars = 2*p,
                                objective_function = obj_expectation,
                                variable_bounds= variable_bounds, 
                                initial_point=pp_optimal_params_B[l])
        print('\nNelderMeadcomplete')
        perturbed_optimal_params, perturbed_optimal_expval = perturbed_optimal_result[0:2] 
        perturbed_params.append(perturbed_optimal_params)
        perturbed_exp_vals.append(perturbed_exp_vals)
    minim_val = np.amin(perturbed_exp_vals)
    minim_index = np.awhere(perturbed_exp_vals == minim_val)[0][0]
    optimal_params_B = perturbed_params[minim_index]
    return [optimal_params_L, optimal_params_B]

In [71]:
qubo_problem = qubo
optimizer = NELDER_MEAD(maxiter = 1000)
initial_freq_params = [ 0.64438965, -0.65847032]
op, offset = qubo_problem.to_ising()
#perturb optimal solution n times:
no_perturb=5
perturb_scale_factor = 0.6
generator = np.random.default_rng()

#Step 1
p = 1
variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
print(f'Step {p}: Building circuit...')
circuit = construct_QAOAcirc(p, op)
print(f'Step {p}: Now optimising parameters (Nelder-Mead)...')
obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) #Define obj_function that inputs only parameters

#Step 1
if p == 1:
    print('initial freq point: '+str(initial_freq_params))
    optimal_result = optimizer.optimize(num_vars = 2*p,
                                        objective_function = obj_expectation,
                                        variable_bounds= variable_bounds, 
                                        initial_point=initial_freq_params)
    optimal_params_B, exp_val_B, no_evals = optimal_result
    optimal_params_L, exp_val_L = optimal_params_B, exp_val_B

Step 1: Building circuit...
Step 1: Now optimising parameters (Nelder-Mead)...
initial freq point: [0.64438965, -0.65847032]
.................................................

In [190]:
#Choose candidate initial parameters for next step
pp_optimal_params_L = np.zeros(shape = (len(optimal_params_L)+2,), dtype = float)+0.05
pp_optimal_params_B_0 = np.zeros(shape = (len(optimal_params_B)+2,), dtype = float)+0.05

pp_optimal_params_L[0:p] = optimal_params_L[0:p]
pp_optimal_params_L[p+1:2*p+1] = optimal_params_L[p:2*p]

pp_optimal_params_B_0[0:p] = optimal_params_B[0:p]
pp_optimal_params_B_0[p+1:2*p+1] = optimal_params_B[p:2*p]

pp_optimal_params_B = np.empty(shape = (no_perturb+1, ), dtype = object)
pp_optimal_params_B[0] = pp_optimal_params_B_0

u_random = np.array([np.array(generator.normal(0, optimal_params_B[i]**2, no_perturb+1)) for i in range(0,p)])
v_random = np.array([np.array(generator.normal(0, optimal_params_B[i]**2, no_perturb+1)) for i in range(p,2*p)])

for j in range(1,len(pp_optimal_params_B)):
    params=np.zeros(shape = (len(optimal_params_B)+2,), dtype=float)+0.05
    params[0:p] = pp_optimal_params_B_0[0:p] + perturb_scale_factor*u_random[:,j]
    params[p+1:2*p+1] = pp_optimal_params_B_0[p+1:2*p+1] + perturb_scale_factor*v_random[:,j]
    pp_optimal_params_B[j] = params
print("\nlocal params: ", pp_optimal_params_L)   
print("perturbed params: ", pp_optimal_params_B)


local params:  [ 0.53923406  0.05194098  0.13882479  0.05       -0.35347069 -0.08898534
 -0.14788939  0.05      ]
perturbed params:  [array([ 0.82212933, -0.94969858,  0.00686471,  0.05      ,  0.21896111,
        0.45164279,  0.05533471,  0.05      ])
 array([ 1.07141374, -0.36610913,  0.0068893 ,  0.05      ,  0.18730822,
        0.60671992,  0.05196011,  0.05      ])
 array([ 1.07086463, -0.92836227,  0.00689655,  0.05      ,  0.21963057,
        0.54527509,  0.05682397,  0.05      ])
 array([ 0.33743131, -0.34655933,  0.00689431,  0.05      ,  0.24186598,
        0.40561724,  0.05360873,  0.05      ])
 array([ 1.24253574, -0.75112342,  0.00687346,  0.05      ,  0.18461433,
        0.5153372 ,  0.05632287,  0.05      ])
 array([ 0.93776878, -0.82641986,  0.0068229 ,  0.05      ,  0.25342176,
        0.49066244,  0.0543737 ,  0.05      ])]


In [191]:
#Step 3
p = 4
optimizer = NELDER_MEAD(maxiter = 2000)
variable_bounds = [ (-np.pi/2, np.pi/2) for _ in range(2*p)]
print(f'Step {p}: Building circuit...')
circuit = construct_QAOAcirc(p, op)
print(f'Step {p}: Now optimising parameters (Nelder-Mead)...')
obj_expectation = lambda x: expectation( circuit, qubo_problem, x ) #Define obj_function that inputs only parameters

#Optimize local
print("Initial: ", pp_optimal_params_L)
optimal_result = optimizer.optimize(num_vars = 2*p,
                        objective_function = obj_expectation,
                        variable_bounds= variable_bounds, 
                        initial_point=pp_optimal_params_L)
optimal_params_L, exp_val_L = optimal_result[0:2]
perturbed_params = [optimal_params_L]
perturbed_exp_vals = [exp_val_L]

optimal_result

Step 4: Building circuit...
Step 4: Now optimising parameters (Nelder-Mead)...
Initial:  [ 0.53923406  0.05194098  0.13882479  0.05       -0.35347069 -0.08898534
 -0.14788939  0.05      ]
........................................................................................................................................................................................................................................

(array([ 0.40496919,  0.05966782,  0.11598603,  0.05328888, -0.28966144,
        -0.10367902, -0.13761269,  0.0616152 ]),
 1.94566,
 232)

In [242]:
l=5
pp_optimal_params_B[l]

array([ 0.93776878, -0.82641986,  0.0068229 ,  0.05      ,  0.25342176,
        0.49066244,  0.0543737 ,  0.05      ])

In [243]:
%%time
print("initial: ", pp_optimal_params_B[l])
perturbed_optimal_result = optimizer.optimize(num_vars = 2*p,
                        objective_function = obj_expectation,
                        variable_bounds= variable_bounds, 
                        initial_point=pp_optimal_params_B[l])
perturbed_optimal_params, perturbed_optimal_expval = perturbed_optimal_result[0:2] 
perturbed_params.append(perturbed_optimal_params)
perturbed_exp_vals.append(perturbed_optimal_expval)
perturbed_optimal_result

initial:  [ 0.93776878 -0.82641986  0.0068229   0.05        0.25342176  0.49066244
  0.0543737   0.05      ]
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................CPU times: user 2min 55s, sys: 578 ms, total: 2min 55s
Wall time: 2min 54s


(array([ 0.7508398 , -1.07267526,  0.00985242,  0.08735804,  0.24008413,
         0.26210454, -0.00524231,  0.04942199]),
 1.76608,
 480)

In [244]:
minim_val = np.amin(perturbed_exp_vals)
minim_index = np.where(perturbed_exp_vals == minim_val)[0][0]
optimal_params_B = perturbed_params[minim_index]

In [263]:
optimal_params_B = perturbed_params[5]

In [264]:
k=4
optimal_parameters = convert_fourier_parameters(k,k,optimal_params_B)
qaoa_circ = QAOA(operator = op, p=k, quantum_instance=BasicAer.get_backend('statevector_simulator'))
optimal_circuit = qaoa_circ.construct_circuit( np.pi*optimal_parameters[0:2*k].astype(float) ) [0]
simulator = Aer.get_backend('statevector_simulator')
result = execute(optimal_circuit, simulator).result()
statevector = Statevector(result.get_statevector())
counts = statevector.sample_counts(shots = 10000)
plot_histogram([counts], figsize = (200,100))

In [265]:
sort_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
sort_counts

[('111001111', 155),
 ('111111001', 130),
 ('111111111', 129),
 ('110110001', 118),
 ('110101001', 112),
 ('111111011', 105),
 ('100100010', 100),
 ('111001001', 96),
 ('110110010', 88),
 ('111111000', 88),
 ('000011010', 87),
 ('001010010', 85),
 ('111101001', 82),
 ('111101110', 77),
 ('111000111', 76),
 ('000110010', 74),
 ('110101111', 73),
 ('101101011', 72),
 ('100110001', 71),
 ('100110010', 71),
 ('110001111', 69),
 ('111100111', 69),
 ('000100100', 67),
 ('101101001', 66),
 ('000010001', 65),
 ('110101110', 65),
 ('000110001', 63),
 ('000100000', 62),
 ('110100111', 61),
 ('000011100', 59),
 ('101110011', 59),
 ('111000000', 59),
 ('100100001', 58),
 ('000010011', 56),
 ('010100110', 56),
 ('000100010', 55),
 ('100100100', 55),
 ('110110110', 55),
 ('111001000', 55),
 ('011101001', 54),
 ('111101101', 54),
 ('000100110', 53),
 ('110001001', 53),
 ('100100000', 52),
 ('101110001', 52),
 ('011111001', 51),
 ('000011000', 50),
 ('011101000', 50),
 ('110111001', 49),
 ('000001110'

In [266]:
counts['001010010']

85

In [269]:
p=3
pp_optimal_params_B_0
u_random = np.array([np.array(generator.normal(0, pp_optimal_params_B_0[i]**2, no_perturb+1)) for i in range(0,p)])
v_random = np.array([np.array(generator.normal(0, pp_optimal_params_B_0[i]**2, no_perturb+1)) for i in range(p+1,2*p+1)])

In [272]:
v_random

array([[-0.02636161,  0.00887011, -0.05002655,  0.078392  ,  0.00488365,
         0.04465729],
       [-0.20760119, -0.14137967,  0.13464426,  0.01795417,  0.17900569,
         0.40072463],
       [-0.00062572,  0.00234111,  0.00827688, -0.00426154, -0.00109213,
         0.00066476]])